In [ ]:
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import re

In [2]:
# Prepara las ociones deseadas para el driver
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

#Inicializa el driver
driver_path = '/Users/saixa/Downloads/chromedriver_win32/chromedriver_win32'
driver = webdriver.Chrome(driver_path, chrome_options=options)

C:\Users\arnau\AppData\Local\Temp\ipykernel_39068\1362028380.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options=options)


In [3]:
def getArguments(driver,location):
    
    #Prepara la ventana
    driver.set_window_position(2000, 0)
    driver.maximize_window()
    time.sleep(1)
   
    # Elimina la primera ventana pop-up 
    WebDriverWait(driver, 3000)\
            .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                            'button._oda838')))\
            .click()
    
    #Abre el buscador de destino
    WebDriverWait(driver, 3000)\
            .until(EC.element_to_be_clickable((By.XPATH,
                                            '/html/body/div[5]/div/div/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/div/div/div/header/div/div[2]/div/div/div/div[1]/div/button[1]')))\
            .click()
    
    # Identifica el input para escribir
    search=driver.find_element(By.ID,"bigsearch-query-location-input")
    sleep(4)
    
    # Escribe el destino
    search.send_keys(location)
    
    #Identifica el botón
    button = driver.find_element(By.XPATH,
                                 '//*[@id="search-tabpanel"]/div/div[5]/div[1]/div[2]/button')
    
    # Activa el click
    button.click()
    


In [4]:
def getAllItemsByPage(pages):
    
    #Inicializa las listas que acumularán todo el contenido del conjunto de datos
    links = []
    content = []
    
    title=[]
    rate = []
    evaluations = []
    superanfitrion=[]
    place = []
    price=[]
    limpieza=[]
    veracidad=[]
    comunicacion=[]
    ubication = []
    llegada=[]
    calidad=[]
    wifi=[]
    tv=[]
    cocina=[]
    dormitorios=[]
    camas=[]
    baños=[]
    
    
    #Itera sobre el número de paginas escogido
    for i in range(0,pages):
        
        # Intenta localizar cada uno de los links en la página de búsqueda y los guarda. Finalmente 
        #acciona el botón siguiente para seguir
        try:
            
            WebDriverWait(driver,3000).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR,"a.ln2bl2p.dir.dir-ltr")))
           
            elems = driver.find_elements(By.CSS_SELECTOR,"a.ln2bl2p.dir.dir-ltr")
            links.extend([elem.get_attribute('href') for elem in elems])
            
            WebDriverWait(driver,3000).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR,"[aria-label='Siguiente']")))
            
            a=driver.find_element(By.CSS_SELECTOR,"[aria-label='Siguiente']").get_attribute('href')
            driver.get(str(a))
            
        # En caso de error sigue con la iteración de paginas
        except:
            continue
    
  

    # Itera de nuevo sobre la lista de links, esta vez de manera completa
    for link in links:
        try :
            driver.get(link)
            sleep(1)
            webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
            # Scrollea hacia abajo para encontrar toda la información necesaria
            sleep(1)
            if len(driver.find_elements(By.XPATH,"//div[contains(., 'Traducción activada')]")) > 0: 
                webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(2)
            else:
                pass
            sleep(1)
            info=driver.find_element(By.XPATH,"//div[contains(., 'Más información sobre las evaluaciones')]").text
            content.append(info)
            
            #Usa expresiones regex para sacar todo el contenido y asignarlo a cada lista
            title.append(re.search(r'b\n(.*?)\n', info).group(1))
            rate.append(re.findall(r'\n(.*?)\n', info)[1][:4])
            evaluations.append([re.findall(r'\n(.*)evaluaciones\n', info)[0][:-1][0] if len(re.findall(r'\n(.*)evaluaciones\n', info)[0][:-1])<5 else 'New'] )
            superanfitrion.append([1 if len(re.findall(r'Superanfitrión', info))>0 else 0])
            place.append(re.findall(r'\n·\n(.*?),(.*?)\n', info)[0])
            price.append(re.findall(r'.\n(.*?)€(.*?)\n', info)[0][0][:-1])
            limpieza.append(re.findall(r'\nLimpieza\n(.*?)\n',info))
            veracidad.append(re.findall(r'\nVeracidad\n(.*?)\n',info))
            comunicacion.append(re.findall(r'\nComunicación\n(.*?)\n',info))
            ubication.append(re.findall(r'[0-9]\nUbicación\n(.*?)\n',info))
            llegada.append(re.findall(r'[0-9]\nLlegada\n(.*?)\n',info))
            calidad.append(re.findall(r'[0-9]\nCalidad\n(.*?)\n',info))
            wifi.append([1 if len(re.findall(r'\n.*?wifi.*?|\n.*?WIFI.*?\n',info))>0 else 0])
            tv.append([1 if len(re.findall(r'\n.*?tv.*?|\n.*?TV.*?\n|\n.*?Televisión.*?|\n.*?Tele.*?\n',info))>0 else 0])
            cocina.append([1 if len(re.findall(r'\n.*?Cocina.*?|\n.*?Kitchen.*?\n',info))>0 else 0])
            dormitorios.append([re.findall(r'· (.*?) dormitorio.*?|· (.*?) Dormitorio.*?', info)[0][0]])
            camas.append([re.findall(r'· [0-9] camas ·|· [0-9] cama ·|· [0-9] Camas ·|· [0-9] Cama ·', info)[0][2] if len(re.findall(r'· [0-9] camas ·|· [0-9] cama ·|· [0-9] Camas ·|· [0-9] Cama ·', info))>0 else re.findall(r'· [0-9] camas ·|· [0-9] cama ·|· [0-9] Camas ·|· [0-9] Cama ·', info)])
            baños.append(re.findall(r'·.*·.*·(.*?)baño',info))
        
        #En caso de algun error durante el proceso sigue con la iteración
        except:
            continue
            
         
 
    return links,content,title,rate,evaluations,superanfitrion,place,price,limpieza,veracidad,comunicacion,ubication,llegada,calidad,wifi,tv,cocina,dormitorios,camas,baños


In [ ]:
driver.get('https://www.airbnb.es/')
getArguments(driver,'Barcelona')
links,content,title,rate,evaluations,superanfitrion,place,price,limpieza,veracidad,comunicacion,ubication,llegada,calidad,wifi,tv,cocina,dormitorios,camas,baños=getAllItemsByPage(3)

In [ ]:
len(rate)

In [ ]:
# Creación de un dataframe en base a las variables que contienen las listas
df = pd.DataFrame({'title':title,
                     'rate':rate,
                     'evaluations':sum(evaluations, []),
                     'superanfitrion':sum(superanfitrion, []),
                     'place':place,
                     'price':price,
                     'limpieza':limpieza,
                     'veracidad':veracidad,
                     'comunicacion':comunicacion,
                     'ubication':ubication,
                     'llegada':llegada,
                     'calidad':calidad,
                     'wifi':sum(wifi, []),
                     'tv':sum(tv, []),
                     'cocina':sum(cocina, []),
                     'dormitorios':sum(dormitorios, []),
                     'camas':sum(camas, [])})

In [ ]:
# Exportar a csv
df.to_csv('airbnb4.csv',index=False)